In [1]:
%%capture
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install umap-learn
!pip install hdbscan
!pip install biopython


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('../data/02_all_signal_peptides/biochemical_result_w_abundance.csv', index_col=False)

In [4]:
df.head()


,gene,n_relative_length,n_kd_hydrophobicity,n_charge_at_ph7,h_relative_length,h_kd_hydrophobicity,h_charge_at_ph7,c_relative_length,c_kd_hydrophobicity,c_charge_at_ph7,abundance
0,AO090005000016-T-p1,4,0.325000,-0.799151,16,1.582353,-0.586943,2,3.400000,-0.590986,0.000000
1,AO090005000029-T-p1,2,-0.650000,-0.775725,10,1.590909,0.442738,10,-0.145455,-1.505143,0.000033
2,AO090005000042-T-p1,2,-1.000000,0.196884,18,1.184211,0.494293,2,-0.833333,-0.827738,0.000048
3,AO090005000053-T-p1,26,0.684615,2.192839,9,1.660000,-0.572083,5,1.116667,-0.147963,0.000000
4,AO090005000059-T-p1,2,-0.650000,-0.775725,13,1.771429,-0.557223,2,1.633333,-0.857574,0.000000


# UMAP calculation

In [5]:
import pandas as pd
import umap.umap_ as umap
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import hdbscan
import seaborn as sns
import matplotlib.colors as mcolors


# Prepare the DataFrame by dropping non-numeric columns
data_for_umap = df.drop(columns=['gene', 'abundance'])

# Standardize the features
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_for_umap)

# Apply UMAP
umap_model = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, random_state=1234)
umap_results = umap_model.fit_transform(scaled_data)

## UMAP clustering with hdbscan

In [ ]:
# Set the aesthetic style of the plots
sns.set(style='white', context='notebook')


# Apply HDBSCAN clustering
clusterer = hdbscan.HDBSCAN(min_cluster_size=50, gen_min_span_tree=True)
cluster_labels = clusterer.fit_predict(umap_results)

# Generating a custom discrete colormap
cmap = plt.cm.Spectral
norm = mcolors.Normalize(vmin=cluster_labels.min(), vmax=cluster_labels.max())

# Visualize the results with clusters
plt.figure(figsize=(10, 8))
scatter = plt.scatter(umap_results[:, 0], umap_results[:, 1], c=cluster_labels, cmap=cmap, norm=norm, s=50, alpha=0.8, edgecolor='k')

# Adding a color bar
plt.colorbar(scatter, label='Cluster', shrink=0.8)

# Enhancing plot aesthetics
plt.title('UMAP Projection with HDBSCAN Clustering', fontsize=16, fontweight='bold')
plt.xlabel('UMAP Component 1', fontsize=14)
plt.ylabel('UMAP Component 2', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.savefig('../data/plots/UMAP_hdbscan_clustering.png', dpi=300, bbox_inches='tight')


plt.show()


: 

## UMAP with K-means

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
import umap.umap_ as umap
from sklearn.preprocessing import StandardScaler

NUMBER_OF_CLUSTERS = 3

# Apply K-means clustering
kmeans = KMeans(n_clusters=NUMBER_OF_CLUSTERS, random_state=1234)  # Adjust the number of clusters as needed
kmeans_clusters = kmeans.fit_predict(umap_results)

# Set the aesthetic style of the plots
sns.set(style='white', context='notebook')

# Create a color palette
palette = sns.color_palette("bright", NUMBER_OF_CLUSTERS)  # Adjust the number of colors based on clusters

# Visualize the results with clusters
plt.figure(figsize=(10, 8))
scatter = plt.scatter(umap_results[:, 0], umap_results[:, 1], c=kmeans_clusters, cmap='viridis', s=50, alpha=0.8, edgecolor='k')

# Adding a color bar
plt.colorbar(scatter, label='Cluster', shrink=0.8)

# Enhancing plot aesthetics
plt.title('UMAP Projection with K-Means Clustering', fontsize=16, fontweight='bold')
plt.xlabel('UMAP Component 1', fontsize=14)
plt.ylabel('UMAP Component 2', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.savefig('../data/plots/UMAP_kmeans_clustering.png', dpi=300, bbox_inches='tight')


plt.show()


: 

In [ ]:
# Add the cluster assignments to your original DataFrame
df['umap_cluster'] = kmeans_clusters

# Now you can see each row with its corresponding cluster
df.head()


: 

In [ ]:

# If you want to see rows belonging to a specific cluster, you can filter the DataFrame
cluster_number = 0  # Change this to the cluster number you're interested in
specific_cluster_df = df[df['umap_cluster'] == cluster_number]
specific_cluster_df.to_csv('../data/02_all_signal_peptides/groups_of_sps/group_data1.csv')

: 

In [ ]:
cluster_number = 1  # Change this to the cluster number you're interested in
specific_cluster_df1 = df[df['umap_cluster'] == cluster_number]
specific_cluster_df1.to_csv('../data/02_all_signal_peptides/groups_of_sps/group_data2')

: 

In [ ]:
cluster_number = 2  # Change this to the cluster number you're interested in
specific_cluster_df2 = df[df['umap_cluster'] == cluster_number]
specific_cluster_df2.to_csv('../data/02_all_signal_peptides/groups_of_sps/group_data3')

: 

## PCA and t-SNE

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

# Assuming 'df' is your DataFrame

# Prepare the DataFrame by dropping the non-numeric column
data_for_analysis = df.drop(columns=['gene', 'abundance'])

# Standardize the features
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_for_analysis)

# Apply PCA
pca = PCA(n_components=2)
pca_results = pca.fit_transform(scaled_data)

# Apply t-SNE
tsne = TSNE(n_components=2, perplexity=30, learning_rate=200, random_state=1234)
tsne_results = tsne.fit_transform(scaled_data)

# Apply K-means clustering
n_clusters = 3  # Adjust based on your data
kmeans = KMeans(n_clusters=n_clusters, random_state=1234)
pca_clusters = kmeans.fit_predict(pca_results)
tsne_clusters = kmeans.fit_predict(tsne_results)

# Set style for plots
sns.set(style='white', context='notebook')

# Plot PCA results
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.scatter(pca_results[:, 0], pca_results[:, 1], c=pca_clusters, cmap='viridis', s=50, alpha=0.6, edgecolor='k')
plt.title('PCA with K-means Clustering', fontsize=14, fontweight='bold')
plt.xlabel('PCA Component 1', fontsize=12)
plt.ylabel('PCA Component 2', fontsize=12)

# Plot t-SNE results
plt.subplot(1, 2, 2)
plt.scatter(tsne_results[:, 0], tsne_results[:, 1], c=tsne_clusters, cmap='viridis', s=50, alpha=0.6, edgecolor='k')
plt.title('t-SNE with K-means Clustering', fontsize=14, fontweight='bold')
plt.xlabel('t-SNE Component 1', fontsize=12)
plt.ylabel('t-SNE Component 2', fontsize=12)

# Adjust layout
plt.tight_layout()

# Save the figure (optional)
plt.savefig('../data/plots/PCA_tSNE_Clustering.png', dpi=300, bbox_inches='tight')

plt.show()


: 

## Comparing clusters

In [ ]:
# Add the t-SNE clusters to your DataFrame
df['tsne_cluster'] = tsne_clusters

: 

In [ ]:
crosstab = pd.crosstab(df['tsne_cluster'], df['umap_cluster'])
crosstab

: 

## Adjusted Rand Index (ARI) or Normalized Mutual Information (NMI)

In [ ]:
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score

# Calculate Adjusted Rand Index
ari = adjusted_rand_score(df['tsne_cluster'], df['umap_cluster'])

# Calculate Normalized Mutual Information
nmi = normalized_mutual_info_score(df['tsne_cluster'], df['umap_cluster'])

print(f"Adjusted Rand Index: {ari:.3f}")
print(f"Normalized Mutual Information: {nmi:.3f}")


: 

## Filtering out gene ids

In [ ]:
martis_list = [
    "AO090003001591-T-p1",
    "AO090005001090-T-p1",
    "AO090023000905-T-p1",
    "AO090038000279-T-p1",
    "AO090701000295-T-p1",
    "AO090003001036-T-p1",
    "AO090001000635-T-p1",
    "AO090005000409-T-p1",
    "AO090009000117-T-p1",
    "AO090102000440-T-p1"
]

# Retaining the rows with these gene IDs
retained_df = df[df['gene'].isin(martis_list)]
retained_df

: 


### Understanding ARI and NMI:

- **Adjusted Rand Index (ARI)**: Measures the similarity of the two assignments, ignoring permutations and with chance normalization. An ARI score of 1 means perfect labeling alignment, while 0 or negative values indicate no better than random chance.

- **Normalized Mutual Information (NMI)**: Also measures the similarity, but it's normalized to account for chance. Like ARI, a score of 1 indicates perfect agreement, and lower values indicate less agreement.


# Looking at the 3 groupings of Signal peptides through DEEPloc

In [ ]:
# Sample DataFrame loading (replace with actual DataFrame loading as needed)
df1_deeploc = pd.read_csv('../data/03_proteomics_data/deeploc_predictions/all_sps/results_646764CA00004786815A8068.csv')
df2_deeploc = pd.read_csv('../data/03_proteomics_data/deeploc_predictions/all_sps/results_6467637F0000328EBD10D7B6.csv')
df3_deeploc = pd.read_csv('../data/03_proteomics_data/deeploc_predictions/all_sps/results_6467641D00003C072DBA1C82.csv')
df_deeploc_data = pd.concat([df1_deeploc, df2_deeploc, df3_deeploc])
df_deeploc_data = df_deeploc_data.rename(columns ={'Protein_ID':'gene'})
df_deeploc_data

: 

In [ ]:
cluster1 = specific_cluster_df.merge(df_deeploc_data,on='gene')
cluster2 = specific_cluster_df1.merge(df_deeploc_data,on='gene')
cluster3 = specific_cluster_df2.merge(df_deeploc_data,on='gene')

: 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Function to get localization counts
def get_localization_counts(df):
    return df['Localizations'].value_counts()

# Get counts for each cluster
counts1 = get_localization_counts(cluster1)
counts2 = get_localization_counts(cluster2)
counts3 = get_localization_counts(cluster3)

# Set the aesthetics for the plots
sns.set(style="whitegrid")
plt.rcParams.update({'font.size': 10, 'font.family': 'sans-serif'})

# Create subplots with larger figure size
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(25, 8), sharey=True)

# Define a color palette with enough colors
max_unique_values = max(counts1.size, counts2.size, counts3.size)
palette = sns.color_palette("husl", max_unique_values)

# Function to create a barplot for a cluster
def create_barplot(ax, counts, title):
    sns.barplot(ax=ax, x=counts.index, y=counts.values, palette=palette[:counts.size])
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.set_xlabel('Localizations', fontsize=12)
    ax.set_ylabel('Count', fontsize=12)
    ax.tick_params(axis='x', rotation=90)  # Rotate labels

# Create bar plots for each cluster
create_barplot(axes[0], counts1, 'Cluster 1 Localizations')
create_barplot(axes[1], counts2, 'Cluster 2 Localizations')
create_barplot(axes[2], counts3, 'Cluster 3 Localizations')

# Adjust layout and spacing
plt.tight_layout()
plt.subplots_adjust(wspace=0.1)  # Adjust horizontal spacing
plt.show()


: 

In [ ]:
cluster1

: 

## Gene ontology of different groups of SPs

In [ ]:
cluster1 = list(specific_cluster_df['gene'])
cluster2 = list(specific_cluster_df1['gene'])
cluster3 = list(specific_cluster_df2['gene'])

: 

These cluster were sent to FUNGIDB and go annotation was made:

In [ ]:
# Sample DataFrame loading (replace with actual DataFrame loading as needed)
df1 = pd.read_csv('../data/14_GO_analysis/hiddenGoEnrichmentResult_group1.tsv', sep='\t')
df2 = pd.read_csv('../data/14_GO_analysis/hiddenGoEnrichmentResult_group2.tsv', sep='\t')
df3 = pd.read_csv('../data/14_GO_analysis/hiddenGoEnrichmentResult_group3.tsv', sep='\t')
df1

: 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# Assuming df1, df2, df3 are your dataframes

def plot_top_bgd_genes(df, title, top_n=10):
    # Sort the dataframe by 'Result count' and select top N
    df_sorted = df.sort_values(by='Result count', ascending=False).head(top_n)

    # Setting the aesthetic style for the plots
    sns.set(style='whitegrid', context='talk', palette='viridis')

    # Plotting
    plt.figure(figsize=(8, 4))
    sns.barplot(x='Result count', y='Name', data=df_sorted, edgecolor='black')
    plt.title(f'Top {top_n} Used Genes in Background for {title}', fontsize=16, fontweight='bold')
    plt.xlabel('Background Gene Count', fontsize=14)
    plt.ylabel('GO Term', fontsize=14)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    sns.despine()  # Removes top and right spines for a cleaner plot

    # Save the figure (optional)
    plt.savefig(f'../data/plots/{title}_Top_Background_Genes.png', dpi=300, bbox_inches='tight')

    plt.show()

# Plotting for each DataFrame
plot_top_bgd_genes(df1, 'Group1')
plot_top_bgd_genes(df2, 'Group2')
plot_top_bgd_genes(df3, 'Group3')


: 

In [ ]:
# Molecular functions for the grouping of signal peptides
df1 = pd.read_csv('../data/14_GO_analysis/group1_mol_function_GO.tsv', sep='\t')
df2 = pd.read_csv('../data/14_GO_analysis/group2_mol_function_GO.tsv', sep='\t')
df3 = pd.read_csv('../data/14_GO_analysis/group3_mol_function_GO.tsv', sep='\t')


: 

In [ ]:
# Plotting for each DataFrame
plot_top_bgd_genes(df1, 'Group1 - molecular functions')
plot_top_bgd_genes(df2, 'Group2 - molecular functions')
plot_top_bgd_genes(df3, 'Group3 - molecular functions')

: 

In [ ]:
synthetic_sps = pd.read_csv('data/15_SecretoGen/Best_signal_peptides_for_A_oryzae_RFP_his_tag_sorted.csv')

: 